### Finding PFAS in groundwater wells downgradient from release
When there is a chemical release on-site the concentration in the subsurface groundwater will be greatest at the point of release and then radiate outward. The rate of flow going out depends on many things including the physiochemical properties of the chemical. Modeling how the chemical moves in the groundwater is very complex; however, one simple question we want to answer is how far does PFAS in the subsurface groundwater move away from the site. 

In this example the program is looking at PFAS compound 'PFOA' at Airport sites only.

To determine this I will output a query from the SQL database with a list of sites with the location of the maximum detected PFAS concentration (which we will assume is the release point). Then the file is input here where the program will look outwards to a range of 4 miles and list every groundwater well where PFAS was detected and the distance. Each tab in the output file is for one Airport and this file can then be brought into a BI software where we can look at trends of decay in concentration vs distance.

In [6]:
import sqlite3
from math import sin, cos, sqrt, atan2, radians
import csv
import xlrd
import xlwt
import xlsxwriter
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
#this program takes a query of airport sites with the maximum detected  PFOA detection well and looks around that well up to 4 miles and exports  all nearby wells into an excel and CSV file for further analysis
#SQL query for reference: SELECT site_name, globalid, lat, long, max(value), date FROM pfas_ca WHERE matrix="Liquid" AND chemical="PFOA" AND value <> 0 AND site_use = "Airport" GROUP BY globalid
R = 6373.0
conn = sqlite3.connect('pfasdb.sqlite')
cur = conn.cursor()

#output to this excel file
writer = pd.ExcelWriter('airport_pfos_gradient.xlsx', engine='xlsxwriter')

#prompt user if they want to upload file
choice = input('Upload File? (answer y or n):')

if choice =="y":
    #user provides filename
    inputfile = input('Enter filename:')
    wb = xlrd.open_workbook(inputfile)
    sheet = wb.sheet_by_index(0)
    #for each row in the file will perform the search
    for row in range(sheet.nrows):
        df = pd.DataFrame(columns = ['globalid', 'locationid', 'distance', 'beginning range','end range','chemical','avg_value','lat','long','date','field_class'])
        site_name = sheet.cell_value(row,0)
        site_id = sheet.cell_value(row,1)
        site_lat = sheet.cell_value(row,2)
        site_long = sheet.cell_value(row,3)
        site_max = sheet.cell_value(row,4)
        site_date = sheet.cell_value(row,5)
        #add the site name and max to the top of the output for each sheet
        airport_info = pd.DataFrame([[site_name,site_date,site_max,"","","","","","","",""]],columns = ['globalid', 'locationid', 'distance', 'beginning range','end range','chemical','avg_value','lat','long','date','field_class'])
        df = df.append(airport_info)

        site_lat = radians(site_lat)
        site_long = radians(site_long)

        #takes the coordinates of the max detect well and measures its distance to each CA well in the database
        for well in cur.execute('SELECT GlobalID, LocationID, Latitude, Longitude, Chemical, Value, ReportingLimit, Date, FieldPtClass FROM ALL_PFAS_CHEMICALS WHERE Matrix="Liquid" AND Chemical="PFOS" AND Value <> 0'):
            #distance_range = range(5)
            range_indicator = 0
            #distance formula for the two sets of coordinates and returns distance in miles
            well_id = well[0]
            well_loc = well[1]
            well_lat = well[2]
            well_long = well[3]
            well_lat_output = well[2]
            well_long_output = well[3]
            chemical = well[4]
            avg_value = well[5]
            date = well[7]
            field_class = well[8]
            if well_id =="" or well_lat =="" or well_long =="" or well_lat ==None or well_long ==None:
                continue
            well_lat = radians(float(well_lat))
            well_long = radians(float(well_long))
            dlon = well_long - site_long
            dlat = well_lat - site_lat
            a = sin(dlat / 2)**2 + cos(well_lat) * cos(site_lat) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c * 0.621371

            #goes through distance from well (0 to 4 miles) in 0.5 mile incremensts and starts outputing them in these categories
            while range_indicator < 4 :
                previous_distance = range_indicator
                range_indicator = range_indicator + 0.5
                if distance <= range_indicator and distance > previous_distance:
                    # information on well that will go to output file
                    well_information = [[well_id,well_loc,distance,previous_distance,range_indicator,chemical, avg_value, well_lat_output, well_long_output, date, field_class]]
                    data = pd.DataFrame([[well_id,well_loc,distance,previous_distance,range_indicator,chemical, avg_value, well_lat_output, well_long_output, date, field_class]], columns = ['globalid', 'locationid', 'distance', 'beginning range','end range','chemical','avg_value','lat','long','date', 'field_class'])
                    df = df.append(data)
                else:
                    continue
        #write all the wells to the output. each sheet is an airport tab that is named as the first 10 characters of the airport name.
        df.to_excel (writer,sheet_name=site_name[0:10])
        #save file
    writer.save()
cur.close()
print("Search Complete")

Upload File? (answer y or n):y
Enter filename:airports_pfos.xls
Search Complete
